In [292]:
!pip3 install mesa

In [293]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

TOTAL_MOVES = 0
shelfX      = 0
shelfY      = 0
#estantepos = [0, 0]

def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Vacuum):
                grid[x][y] = 7
            elif isinstance(content, Floor):
                if content.live >= 1:
                    grid[x][y] = content.numBoxes
                else:
                    grid[x][y] = 0
            #else:
            #    grid[x][y] = 0
    return grid

In [294]:
class RandomWalker(Agent):

    grid = None
    x = None
    y = None
    moore = True
    

    def __init__(self, unique_id, pos, model, moore=False):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore

    def random_move(self):
        # Pick the next cell from the adjacent cells.
        #next_moves = self.model.grid.get_neighborhood(self.pos, self.moore, include_center = False)
        #print(next_moves)
        #next_move = self.random.choice(next_moves)

        neighbors  = self.model.grid.get_neighbors(self.pos, self.moore, include_center = False)
        aux = [obj for obj in neighbors if not isinstance(obj, Vacuum)]
        #print("neighbors")
        #print(neighbors)
        #print("aux")
        #print(aux)

        # Now move:
        if len(aux) > 0:
          next_move = self.random.choice(aux)
          self.model.grid.move_agent(self, next_move.pos)
        else:
          self.model.grid.move_agent(self, self.pos)


    def move_to_shelf(self):
        global shelfX
        global shelfY
        print(shelfX)
        print(shelfY)
        estantepos=(shelfX, shelfY)
        #muevete al estante
        next_move = self.pos

        #estantepos=(shelfX,shelfY)
        if next_move[0] > estantepos[0]: #Si el estante esta hacia arriba
          next_move = (next_move[0]-1,next_move[1])
        elif next_move[0] < estantepos[0]:
          next_move = (next_move[0]+1,next_move[1])
        elif next_move[1] > estantepos[1]: #Si el estante esta hacia la izquierda
          next_move = (next_move[0],next_move[1]-1)
        elif next_move[1] < estantepos[1]: #Si el estante esta hacia la derecha
          next_move = (next_move[0],next_move[1]+1)
        

        neighbors  = self.model.grid.get_neighbors(self.pos, self.moore, include_center = False)
        aux = [obj for obj in neighbors if not isinstance(obj, Vacuum)]

        if len(aux) > 0:
          self.model.grid.move_agent(self, next_move)
        else:
          self.model.grid.move_agent(self, self.pos)
        #self.model.grid.move_agent(self, next_move)
        

        


In [295]:
class Vacuum(RandomWalker):
    def __init__(self, unique_id, pos, model, moore):
        super().__init__(unique_id, pos, model, moore=moore)
        self.live=7
        self.load=False
        #self.moves=0

    def step(self):
        global shelfX
        global shelfY
        global TOTAL_MOVES
        estantepos = (shelfX, shelfY)

        if self.load == False:
          self.random_move()
          TOTAL_MOVES = TOTAL_MOVES + 1
                  
          #Si hay caja, agarra caja
          #x, y = self.pos
          this_cell = self.model.grid.get_cell_list_contents([self.pos])
          dirt = [obj for obj in this_cell if isinstance(obj, Floor) and obj.numBoxes == 1]
          if len(dirt) > 0 and self.pos != estantepos:
              dirt_to_eat = self.random.choice(dirt)

              # Elimina suciedad
              dirt_to_eat.live = 0
              dirt_to_eat.numBoxes = 0
              #dirt_to_eat.pos = estantepos
              self.load = True
              #print(sheep_to_eat.live)
        else:
          #muevete al estante incompleto
          if self.pos != estantepos: 
            self.move_to_shelf();
            TOTAL_MOVES = TOTAL_MOVES + 1
          else:
            #checa si esta completo
            '''
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            dirt = [obj for obj in this_cell if isinstance(obj, Floor)]
            print("dirt: ")
            print(dirt)
            print("dirt[0].numBpexes: ")
            print(dirt[0].numBoxes)
            if self.pos == estantepos and dirt[0].numBoxes > 4 : #Si estoy en el estante... haz esto...
              print("entre al primer if")
              if estantepos[1] + 1 < width:
                print("entre al segundo if")
                #global shelfY
                shelfY = shelfY + 1
                next_move = (next_move[0],next_move[1]+1)
                self.model.grid.move_agent(self, next_move)
              elif estantepos[0] + 1 < height:
                print("entre al tercer if")
                #global shelfX
                shelfX = shelfX + 1
                next_move = (next_move[0]+1,next_move[1])
                self.model.grid.move_agent(self, next_move)
                '''

            #coloca caja
            if self.pos == estantepos:
              this_cell = self.model.grid.get_cell_list_contents([self.pos])
              dirt = [obj for obj in this_cell if isinstance(obj, Floor)] #>=1
              #print(dirt)
              if dirt[0].numBoxes > 4: #Cuando ya estoy lleno, haz esto...
                print("entre al primer if")
                if estantepos[1] + 1 < width:
                  print("entre al segundo if")
                  #global shelfY
                  shelfX = shelfX + 1
                  #next_move = (next_move[0],next_move[1]+1)
                  #self.model.grid.move_agent(self, next_move)
                elif estantepos[0] + 1 < height:
                  print("entre al tercer if")
                  #global shelfX
                  shelfY = shelfY + 1
                  #next_move = (next_move[0]+1,next_move[1])
                  #self.model.grid.move_agent(self, next_move)
              if dirt[0].numBoxes < 5: #Solo para verificar que exista algo de tipo piso ahi
                new_box = dirt[len(dirt)-1]
                new_box.live=1;
                new_box.numBoxes+=1;

              self.load=False
                
class Floor(Agent):
    '''
    Representa a un agente o una celda con estado vivo (1) o muerto (0)
    '''
    def __init__(self, unique_id, pos, live, numBoxes, model):
        '''
        Crea un agente con estado inicial aleatorio de 0 o 1, también se le asigna un identificador 
        formado por una tupla (x,y). También se define un nuevo estado cuyo valor será definido por las 
        reglas mencionadas arriba.
        '''
        super().__init__(unique_id, model)
        self.live = live
        self.numBoxes = numBoxes;
        self.pos=pos
        self.next_state = None
    
    def step(self):
        this_cell = self.model.grid.get_cell_list_contents([self.pos])

In [296]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


class CleaningModel(Model):
    def __init__(self, width, height, initial_dirt, initial_bot):
        #self.num_agents = width * height
        self.initial_dirt=initial_dirt
        self.initial_bot=initial_bot
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        
        
        
        for i in range(width):
          for j in range(height):
            dirt = Floor(i*height+j,(i, j), 0, 0, self)
            self.grid.place_agent(dirt, (i, j))
            self.schedule.add(dirt)

        #spawn CAJAS
        for i in range(self.initial_dirt):
            x = self.random.randrange(width)
            y = self.random.randrange(height)

            this_cell = self.grid.get_cell_list_contents([(x, y)])
            dirt = [obj for obj in this_cell if isinstance(obj, Floor) and obj.numBoxes >= 1]

            while len(dirt) >= 1 or x == 0 and y == 0:
              x = self.random.randrange(width)
              y = self.random.randrange(height)

              this_cell = self.grid.get_cell_list_contents([(x, y)])
              dirt = [obj for obj in this_cell if isinstance(obj, Floor) and obj.numBoxes >= 1]
              
            if len(dirt) <= 0:
              dirt = Floor(i+(width*height),(x, y), 1, 1, self)
              self.grid.place_agent(dirt, (x, y))
              self.schedule.add(dirt)

        #spawn aspiradoras
        for i in range(self.initial_bot):
          x = self.random.randrange(width)
          y = self.random.randrange(height)

          this_cell = self.grid.get_cell_list_contents([(x, y)])
          dirt = [obj for obj in this_cell if isinstance(obj, Floor) and obj.numBoxes >= 1]
          vac  = [obj for obj in this_cell if isinstance(obj, Vacuum)]

          while len(dirt) >= 1 or len(vac) >= 1 or x == 0 and y == 0:
            print("while")
            x = self.random.randrange(width)
            y = self.random.randrange(height)

            this_cell = self.grid.get_cell_list_contents([(x, y)])
            dirt = [obj for obj in this_cell if isinstance(obj, Floor) and obj.numBoxes >= 1]
            vac  = [obj for obj in this_cell if isinstance(obj, Vacuum)]
            
          if len(dirt) <= 0 and len(vac) <= 0:
            bot = Vacuum(i+initial_dirt+(width*height), (x, y), self, False)
            self.grid.place_agent(bot, (x, y))
            self.schedule.add(bot)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    '''def isallclean(self):
        filthy_cells = 0
        for cell in model.grid.coord_iter():
            cell_content, x, y = cell
            for content in cell_content:
                if isinstance(content, Floor):
                    if content.live == 1:
                        filthy_cells = filthy_cells + 1
                
        if filthy_cells>=1:
            #print("hay sucio")
            return False
        else:
            #print("no hay sucio")
            return True
        '''
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.schedule.step()
        self.datacollector.collect(self)
        

In [297]:
#Datos Iniciales
TIEMPO_MAXIMO_EJECUCION = 1.0 #Tiempo Maximo de Ejecucion
initial_bots            = 5 #Numero de Agentes
initial_dirt            = 20
width                   = 10 #M
height                  = 10 #N
NUM_GENERATIONS         = 0

#cont=0
start_time = time.time()
tiempo_inicio=str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
model = CleaningModel(width, height, initial_dirt, initial_bots)
while((time.time()-start_time) < TIEMPO_MAXIMO_EJECUCION and NUM_GENERATIONS <= 250):
    #print(cont)
    model.step()
    NUM_GENERATIONS += 1

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))



while
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
entre al primer if
entre al segundo if
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
entre al primer if
entre al segundo if
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
2
0
entre al primer if
entre al segundo if
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
3
0
Tiempo de ejecución: 0:00:00.219093


In [298]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(type(all_grid))
print(all_grid)

<class 'pandas.core.frame.DataFrame'>
                                                  Grid
0    [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 7.0,...
1    [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...
2    [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...
3    [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,...
4    [[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 7.0, 0.0, 0.0,...
..                                                 ...
246  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
247  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
248  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
249  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
250  [[5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

[251 rows x 1 columns]


In [299]:
%%capture
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [300]:
anim

In [301]:
TOTAL_MOVES

1235

In [302]:
data = ["infy", "tcs", "affle", "dixon", "astral"] 

last_element = data[-1]
print(last_element)

for i in range(9):
    for j in range(8):
        print(i * 8 + j, end=" ") 
        # print(f'{matrix[i * size + j]:>2d}', end=" ") to format 2 digit numbers
    print()
  
pruebas_s = (0, 0)
pruebas_23 = (0 , 3)
pruebas_s = pruebas_23
pruebas_s

astral
0 1 2 3 4 5 6 7 
8 9 10 11 12 13 14 15 
16 17 18 19 20 21 22 23 
24 25 26 27 28 29 30 31 
32 33 34 35 36 37 38 39 
40 41 42 43 44 45 46 47 
48 49 50 51 52 53 54 55 
56 57 58 59 60 61 62 63 
64 65 66 67 68 69 70 71 


(0, 3)

In [303]:
x = 0
def f():
    global x
    x = 1
f()
print(x) #=>1

print(shelfX)
print(shelfY)

1
3
0
